## Cobol Tool

Requisitos:  `pip install coboljsonifier`

In [1]:
import cobol_funcion as cf

interface = 'employee_data.bin'
archivo_cobol = 'inbound.cob'

In [2]:
cf.detectar_encoding(interface)

Encoding detectado: Windows-1252


'Windows-1252'

In [2]:
cf.read_file(interface)

b'1234567890John Doe\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x1e\x00\x00\x00\xc0PCG0987654321Jane Smith\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x19\x00\x00\x00\x80`jG'


In [9]:
# Se valida el Formato del archivo cobol
df = cf.extractor_copybook(archivo_cobol)
df

,Name,type,format,size,decimals
0,EMP-ID,ALPHANUMERIC,X(10),10,0
1,EMP-NAME,ALPHANUMERIC,X(30),30,0
2,EMP-AGE,NUMERIC,99,2,0
3,EMP-SALARY,NUMERIC,9(5)V99,7,2


In [ ]:
# Use book structure to build a parser (FLAT_ASCII / BINARY_EBCDIC)
rows = 2

In [ ]:
bookfname = 'bgdtacu.cob'
interface = 'BGDTACU-20230727.DAT'


dict_structure = CopybookExtractor(bookfname).dict_book_structure
df = pd.DataFrame([key for key in dict_structure.keys()], columns=['Name'])
df['type'] = [value['type'] for value in dict_structure.values()]
df['format'] = [value['format'] for value in dict_structure.values()]
df['size'] = [value['size'] for value in dict_structure.values()]
df['decimals'] = [value['decimals'] for value in dict_structure.values()]

parser = Parser(dict_structure, ParseType.BINARY_EBCDIC).build()
size = parser.size
print("// Registry calculated lenght:", size)

<hr>

In [2]:
import json
from coboljsonifier.copybookextractor import CopybookExtractor
from coboljsonifier.parser import Parser
from coboljsonifier.config.parser_type_enum import ParseType
import pandas as pd
import ast

pd.options.display.max_rows = 4000

### PARSE ASCII

In [3]:
bookfname = 'cobol.cob'
dict_structure = CopybookExtractor(bookfname).dict_book_structure

In [4]:
# Use book structure to build a parser (FLAT_ASCII / BINARY_EBCDIC)
parser = Parser(dict_structure, ParseType.FLAT_ASCII).build()

data= '00710241234512ABC12345612'
parser.parse(data)
dictvalue = parser.value
pretty = json.dumps(dictvalue, indent=4,default=str)
val_convert = ast.literal_eval(pretty.replace("null", "None"))

In [5]:
df = pd.DataFrame([key for key in dict_structure.keys()], columns=['Name'])
df['type'] = [value['type'] for value in dict_structure.values()]
df['format'] = [value['format'] for value in dict_structure.values()]
df['size'] = [value['size'] for value in dict_structure.values()]
df['decimals'] = [value['decimals'] for value in dict_structure.values()]
df['result'] = [value for value in val_convert.values()]
df

,Name,type,format,size,decimals,result
0,CLAIM-NUMBER,ALPHANUMERIC,X(3),3,0,007
1,ADMISSION-DATE,NUMERIC,S9(4),4,0,1024
2,FULL-DAYS,NUMERIC,S9(5),5,0,12345
3,INTERMEDIARY-NUM,NUMERIC,9(2),2,0,12
4,PROVIDER,ALPHANUMERIC,X(3),3,0,ABC
5,INPATIENT-DED,NUMERIC,S9(4)V99,6,2,1234.56
6,PATIENT-STATUS,ALPHANUMERIC,X(2),2,0,12


### PARSE BINARY

In [ ]:
bookfname = 'bgdtacu.cob'
interface = 'BGDTACU-20230727.DAT'
rows = 2

dict_structure = CopybookExtractor(bookfname).dict_book_structure
df = pd.DataFrame([key for key in dict_structure.keys()], columns=['Name'])
df['type'] = [value['type'] for value in dict_structure.values()]
df['format'] = [value['format'] for value in dict_structure.values()]
df['size'] = [value['size'] for value in dict_structure.values()]
df['decimals'] = [value['decimals'] for value in dict_structure.values()]

In [ ]:
parser = Parser(dict_structure, ParseType.BINARY_EBCDIC).build()
size = parser.size
print("// Registry calculated lenght:", size)

with open(interface, 'rb') as f2:
    for x in range(rows):

        # EBCDIC
        data = f2.read(size)
        if not data:
            break
           
        if data[0:2] == b'\xF0\xF0':    # ====> El nuestro es este formato
            parser.parse(data)
            dictvalue = parser.value

            pretty = json.dumps(dictvalue, indent=4,default=str)
            val_convert = ast.literal_eval(pretty.replace("null", "None"))
            col = 'col' + str(x)
            df[col] = [value for value in val_convert.values()]

        else:
            print(f'// Registry type {data[0:2]} not processed')

### Posibles errores

En **Extracting copybook structure** le da error los  |    PIC X.   => Cambiarlos por =>   PIC X(1).  en el copybook